In [ ]:
import openai
import speech_recognition as sr
import time
import subprocess
from gtts import gTTS
import tempfile
from dotenv import load_dotenv
import os

load_dotenv()

# Function to speak text using gTTS
def speak_text(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as temp_audio:
        tts.save(temp_audio.name)
        subprocess.call(["ffplay", "-nodisp", "-autoexit", temp_audio.name])  # Example with ffplay
        # Or subprocess.call(["aplay", temp_audio.name])  # Example with aplay

def transcribe_audio_to_text(filename):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)
    try:
        return recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        print('Google Speech Recognition could not understand audio')
        return ""
    except sr.RequestError as e:
        print(f'Could not request results from Google Speech Recognition service; {e}')
        return ""

def generate_response(user_question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_question}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=400,
        n=1,
        stop=None,
        temperature=0.5,
    )
    return response["choices"][0]["message"]["content"]

def main():
    while True:
        # Wait for user to say "Assistant"
        print("Say 'okay assistant' to start engaging with the assistant...")
        with sr.Microphone() as source:
            recognizer = sr.Recognizer()
            audio = recognizer.listen(source)
            try:
                transcription = recognizer.recognize_google(audio)
                if transcription.lower() == "okay assistant":
                    # Record Audio
                    filename = "input.wav"
                    print("What is your question...")
                    with sr.Microphone() as source:
                        recognizer = sr.Recognizer()
                        source.pause_threshold = 1
                        audio = recognizer.listen(source, phrase_time_limit=None, timeout=None)
                        with open(filename, "wb") as f:
                            f.write(audio.get_wav_data())
                            
                    # Transcribe audio to text
                    text = transcribe_audio_to_text(filename)
                    if text:
                        print(f"You said: {text}")
                        
                        response = generate_response(text)
                        print(f"Assistant's Response: {response}")
                        
                        # Read response using text-to-speech
                        speak_text(response)
            except Exception as e:
                print("An error occurred: {}".format(e))
                
if __name__ == "__main__":
    # Set your OpenAI API key
    openai.api_key = os.getenv("OPENAI_API_KEY")
    main()
